In [396]:
# Réécriture avec commentaires INPUT / OUTPUT restaurés

import plotly.graph_objects as go
import pandas as pd
from IPython.display import display, HTML

# === CLASSES ===
class TimeSeries:
    def __init__(self, hourly_values):  # INPUT: liste de 24 valeurs horaires (ex: demande ou PV)
        self.hourly_values = hourly_values
    def __getitem__(self, hour):  # OUTPUT: valeur à l'heure demandée (modulo 24)
        return self.hourly_values[hour % 24]

class Battery:
    def __init__(self, battery_capacity_kWh, initial_soc_ratio):  # INPUT: capacité batterie (kWh), SoC initial (%)
        self.capacity_kWh = battery_capacity_kWh
        self.initial_soc_ratio = initial_soc_ratio
        self.current_energy_kWh = initial_soc_ratio * battery_capacity_kWh  # OUTPUT: énergie disponible actuelle (kWh)
    def charge(self, charge_kWh):  # INPUT: demande de charge (kWh) → OUTPUT: charge réelle effectuée (kWh)
        possible_charge = min(charge_kWh, 11, self.capacity_kWh - self.current_energy_kWh)
        self.current_energy_kWh += possible_charge
        return possible_charge
    def discharge(self, house_demand_kWh, min_soc_ratio):  # INPUT: demande maison (kWh), SoC min autorisé (%)
        max_discharge_possible = self.current_energy_kWh - min_soc_ratio * self.capacity_kWh
        energy_supplied = min(house_demand_kWh, 11, max_discharge_possible)  # OUTPUT: énergie réellement fournie
        self.current_energy_kWh -= energy_supplied
        return energy_supplied
    def get_soc_percent(self):  # OUTPUT: SoC en pourcentage actuel
        return round((self.current_energy_kWh / self.capacity_kWh) * 100, 2)

class Vehicle:
    def __init__(self, battery, arrival_hour, departure_hour, target_soc_ratio):  # INPUT: batterie, heure d’arrivée, départ, SoC cible
        self.battery = battery
        self.arrival_hour = arrival_hour
        self.departure_hour = departure_hour
        self.target_soc_ratio = target_soc_ratio

# === PARAMÈTRES ===
BATTERY_CAPACITY_KWH = 70
MAX_CHARGE_DISCHARGE_KWH_PER_HOUR = 11
MIN_SOC_RATIO = 0.2

# === DONNÉES ENTRÉE (INPUT) ===
house_demand = TimeSeries([0.5]*6 + [1.0]*2 + [2.5]*4 + [1.5]*4 + [2.0]*6 + [0.5]*2)  # INPUT: demande maison par heure
pv_generation = TimeSeries([0]*6 + [1.0]*3 + [2.5]*5 + [1.5]*4 + [0.5]*6)  # INPUT: production PV par heure
battery = Battery(battery_capacity_kWh=BATTERY_CAPACITY_KWH, initial_soc_ratio=0.4)
vehicle = Vehicle(battery=battery, arrival_hour=17, departure_hour=15, target_soc_ratio=0.8)

# === SIMULATION (OUTPUT principaux : SoC, battery flow, net load) ===
soc_each_hour_kWh = [None] * 24
battery_flow_kWh = [0] * 24
net_load_kWh = [0] * 24
current_soc_kWh = battery.current_energy_kWh

peak_hours = [7, 8, 18, 19]
off_peak_hours = [h for h in range(24) if h not in peak_hours]

if vehicle.arrival_hour < vehicle.departure_hour:
    hours_range = range(vehicle.arrival_hour, vehicle.departure_hour)
else:
    hours_range = range(vehicle.arrival_hour, vehicle.departure_hour + 24)

for h in hours_range:
    h_mod = h % 24
    pv = pv_generation[h_mod]
    demand = house_demand[h_mod]
    battery_effect = 0

    if h == vehicle.arrival_hour:
        soc_each_hour_kWh[h_mod] = current_soc_kWh
        continue

    if h_mod in peak_hours:
        house_need = demand - pv
        max_available = current_soc_kWh - MIN_SOC_RATIO * BATTERY_CAPACITY_KWH
        power_possible = min(MAX_CHARGE_DISCHARGE_KWH_PER_HOUR, max_available)

        soc_target_kWh = vehicle.target_soc_ratio * BATTERY_CAPACITY_KWH
        remaining_hours = (vehicle.departure_hour - h_mod) % 24
        max_future_charge = remaining_hours * MAX_CHARGE_DISCHARGE_KWH_PER_HOUR
        soc_margin_percent = 100 * current_soc_kWh / BATTERY_CAPACITY_KWH

        if (current_soc_kWh + max_future_charge < soc_target_kWh) and \
           (remaining_hours < 4 or soc_margin_percent < 60):
            battery_effect = 0
        else:
            if power_possible >= house_need:
                battery_effect = -house_need
            elif power_possible > 0:
                battery_effect = -power_possible

        current_soc_kWh += battery_effect

    elif h_mod in off_peak_hours:
        if current_soc_kWh < vehicle.target_soc_ratio * BATTERY_CAPACITY_KWH:
            available_pv = max(pv - demand, 0)
            energy_needed = vehicle.target_soc_ratio * BATTERY_CAPACITY_KWH - current_soc_kWh
            max_charge = min(MAX_CHARGE_DISCHARGE_KWH_PER_HOUR, energy_needed)

            battery_effect = min(max_charge, available_pv)
            if battery_effect <= 0:
                battery_effect = max_charge

            current_soc_kWh += battery_effect

    soc_each_hour_kWh[h_mod] = current_soc_kWh
    battery_flow_kWh[h_mod] = round(battery_effect, 2)
    net = demand - pv + battery_effect
    net_load_kWh[h_mod] = round(max(net, 0), 2)

# Forcer SoC à l'heure de départ
soc_each_hour_kWh[vehicle.departure_hour] = current_soc_kWh

# === VISUALISATION (OUTPUT interactif) ===
soc_percent = [round(100 * s / BATTERY_CAPACITY_KWH, 2) if s is not None else None for s in soc_each_hour_kWh]
hours = [f"{h:02d}:00" for h in range(24)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=hours, y=pv_generation.hourly_values, name="PV (kW)", mode="lines+markers", line=dict(color="orange"), yaxis="y1"))
fig.add_trace(go.Scatter(x=hours, y=house_demand.hourly_values, name="House Demand (kW)", mode="lines+markers", line=dict(color="gray"), yaxis="y1"))
fig.add_trace(go.Scatter(x=hours, y=net_load_kWh, name="Net Load (kW)", mode="lines+markers", line=dict(color="black"), yaxis="y1"))
fig.add_trace(go.Scatter(x=hours, y=battery_flow_kWh, name="Battery Flow (kWh)", mode="lines+markers", line=dict(color="brown", dash="dot"), yaxis="y1"))
fig.add_trace(go.Scatter(x=hours, y=soc_percent, name="SoC (%)", mode="lines+markers", line=dict(color="blue", dash="dot"), yaxis="y2", connectgaps=False))
fig.add_trace(go.Scatter(x=[hours[vehicle.arrival_hour]], y=[round(vehicle.battery.initial_soc_ratio * 100, 2)], mode='markers', marker=dict(color='green', size=12), name="Arrival", yaxis="y2"))
fig.add_trace(go.Scatter(x=[hours[vehicle.departure_hour]], y=[round(current_soc_kWh * 100 / BATTERY_CAPACITY_KWH, 2)], mode='markers', marker=dict(color='red', size=12), name="Departure", yaxis="y2"))

fig.update_layout(
    title="<b>✅ V2H Simulation",
    xaxis=dict(title="Hour"),
    yaxis=dict(title="Power (kW)", side="left"),
    yaxis2=dict(title="State of Charge (%)", overlaying="y", side="right", range=[0, 100]),
    height=650,
    template="plotly_white",
    legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="center", x=0.5),
    margin=dict(t=100, b=80, l=60, r=60)
)

fig.show()




# === NOUVEL AFFICHAGE ÉVÉNEMENTS IMPORTANT (ultra lisible) ===
summary_events = []

for h in range(24):
    soc = soc_each_hour_kWh[h]
    pv = pv_generation[h]
    demand = house_demand[h]
    surplus = pv - demand > 0

    can_reach_target = current_soc_kWh >= vehicle.target_soc_ratio * BATTERY_CAPACITY_KWH
    soc_margin = soc is not None and (soc - MIN_SOC_RATIO * BATTERY_CAPACITY_KWH) >= 0

    events = []

    if surplus:
        events.append("☀️ Surplus PV")
    if not soc_margin:
        events.append("⚠️ Cannot Discharge")
    if not can_reach_target and h == vehicle.departure_hour:
        events.append("⚠️ SoC target not reached")

    if events:
        summary_events.append({
            "Hour": f"{h:02d}:00",
            "Event(s)": ", ".join(events)
        })

# Affichage propre
if summary_events:
    summary_df = pd.DataFrame(summary_events)
    display(HTML(summary_df.to_html(index=False, escape=False)))
else:
    display(HTML("<b style='color:green;'>✅ No major events detected today.</b>"))



Hour,Event(s)
12:00,☀️ Surplus PV
13:00,☀️ Surplus PV
16:00,⚠️ Cannot Discharge
